[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MatteoFalcioni/Learning-LangGraph/blob/main/notebooks/11_supervisor.ipynb)

### Setup

#### Install requirements

In [2]:
%pip install -q -U -r https://raw.githubusercontent.com/MatteoFalcioni/Learning-LangGraph/main/requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### local (notebooks or files)

In [3]:
from dotenv import load_dotenv
load_dotenv()  # load api keys

True

#### Colab

In [4]:
from google.colab import userdata
import os

REQUIRED_KEYS = [
    'OPENAI_API_KEY',
    'LANGSMITH_TRACING',
    'LANGSMITH_ENDPOINT',
    'LANGSMITH_API_KEY',
    'LANGSMITH_PROJECT'
]

def _set_colab_keys(key : str):
    # Retrieve the secret value using its key/name
    secret_value = userdata.get(key)
    # set it as a standard OS environment variable
    os.environ[key] = secret_value

for key in REQUIRED_KEYS:
    _set_colab_keys(key)

ModuleNotFoundError: No module named 'google.colab'

# Supervised Workflows & MCP

In this notebook we will be creating a supervised agent workflow: a supervisor agent managing 3 subagents.

In our example our subagents will peform three distinct functions: 
- the `email_agent` will be checking incoming mails on our Gmail, writing drafts, requesting for approval or edits (HITL) and sanding emails if approved;
- the `calendar_agent` will check our Google Calendar, remind us of incoming events and writing new events if requested;
- the `github_agent` will check our `GitHub`notifications and, if we approve, dismiss them.


<center>
    <img src="../projects/supervisor/src/graph/graph_plot/supervised_20251229_163528.png" width="600">
<center>

We will be writing python files now instead of notebooks, and use this notebook only for notes and remarks on our implementation. 

Check out the full implementation here: [`projects/supervisor`](../projects/supervisor/).

# 1. Handoff Tools

We start with the core of the implementation of a supervisor agent: handing off tasks to the subagents. 

In order to do so we usually follow a simple approach: give our supervisor agent "handoff tools", i.e. tools that it can call to route the flow to the other agents. 

> **Note 1:** You'll see that i usually use a simple agent for the supervisor, while i construct nodes for the subagents. 
>
>This is not mandatory. I just like to do it because usually the subagents need to perform more complex tasks, and therefore I often need to do additional operations in the subagent nodes.

---



> **Note 2 (Advanced):** Following this logic, if we need to perform additional operations when dealing with the supervisor, we can create a supervisor node as well. 
>
> Catch for this second approach: if the supervisor **routes** with a tool (using Command) but the supervisor is invoked in a node, the goto must be propagated. In this we could extract the goto from the Command returned from the supervisor agent, or maybe it's easier to just use a state var "next" that the supervisor fills with its tools. Or even a structured output.

---

Let's see some examples of handoff tools. 

Note that all these tools are structured with `graph.PARENT` because we do not have a supervisor node right now. Our supervisor will be running as a compiled agent inside a parent graph.

Therefore we need `Command.PARENT` to tell the `Command` that it should escape the supervisor and navigate to that given node in the parent graph.

```
Parent Graph
├── supervisor_node  # (our supervisor agent)
│       └── handoff_to_gmail_agent tool (returns Command)
└── gmail_agent_node
```

### 1.1. Simple Handoff Tool

In [ ]:
from typing import Annotated
from langchain.tools import tool, ToolRuntime
from langchain_core.messages import ToolMessage, HumanMessage
from langgraph.types import Command, interrupt

agent_name = "gmail_agent"
@tool
def handoff_to_gmail_agent(
    runtime: ToolRuntime,
) -> Command:
    """
    Transfer the task to the gmail agent.
    """

    tool_msg = ToolMessage(
        content=f"Successfully transferred to subagent",
        tool_call_id=runtime.tool_call_id,
    )

    return Command(
        goto=agent_name,
        update={"messages": [tool_msg]},
        graph=Command.PARENT,
    )

### 1.2. Handoff Tool with Task

We could add a task message from the supervisor to the full chat history, or we could replace chat history with just the task. 

These are two different approaches: both make the agent focus more on the task, but the latter makes the agent focus specifically on that task the supervisor asks and erases the subagent memory of everything else. This is the approach of `deepagents` for subagents. Let's see how: 

In [ ]:
agent_name = "gmail_agent"
@tool
def handoff_to_gmail_agent(
    task: Annotated[str, "The task to be performed by the subagent"],
    runtime: ToolRuntime,
) -> Command:
    """
    Transfer the task to the gmail agent.
    """

    tool_msg = ToolMessage(
        content=f"Successfully transferred to subagent",
        tool_call_id=runtime.tool_call_id,
    )

    state = runtime.state
    task_msg = HumanMessage(content=task)

    return Command(
        goto=agent_name,
        update={**state, "messages": [tool_msg] + [task_msg]},  # this will replace the previous chat history
        graph=Command.PARENT,  # transfer to nearest parent graph 
    )

### 1.3. Handoff Tool with HITL

We can add human approval before routing:

In [ ]:
agent_name = "gmail_agent"
@tool
def handoff_to_gmail_agent_HITL(
    task: Annotated[str, "The task to be performed by the subagent"],
    runtime: ToolRuntime,
) -> Command:
    """
    Transfer the task to the gmail agent.
    """
    # interrupt the main flow and ask the user for approval
    usr_response = interrupt(
            value=f"The agent supervisor wants to call the {agent_name} to perform the following task: *{task}*\nDo you approve?"
        )

    if usr_response['decision'] == "accept":
            goto = agent_name
            tool_msg = [ToolMessage(content=f"Successfully transferred to {agent_name}", tool_call_id=runtime.tool_call_id)]
            task_msg = [HumanMessage(content=f"The agent supervisor advices you to perform the following task : \n{task}")]
            msgs = tool_msg + task_msg

    elif usr_response['decision'] == 'reject':
        goto = "supervisor"
        msgs = [ToolMessage(content=f"Routing to {agent_name} was rejected by the user.", tool_call_id=runtime.tool_call_id)]
    
    else: 
        raise ValueError(f"Invalid user response: {usr_response['decision']}")

    return Command(
        goto=goto,
        update={"messages": msgs}, 
        graph=Command.PARENT,  
    )

## 2. Model Context Protocol (*MCP*)

For the GitHub tool we will use ***Model Context Protocol***, a new technology that is gaining a lot of traction in agentic AI applications. 

It's soon referred to as the "USB C" of AI agents: a comparison that's kind of inaccurate, since the MCP technology is often propietary, can give rise to technical overhead (it can introudce additional latency) and can have security risks if not handled carefully. 

Still, MCP is revolutionary for AI agents: Before MCP, if you had 5 AI models and 5 data sources (Google Drive, Slack, GitHub, etc.), you had to write 25 different integrations ($5 \times 5$).MCP changes this to a "plug-and-play" model ($M + N$ complexity).

Now, MCP are widely used in no code / low code AI agents builds: they are basically standardized tools that are plugable and ready to use. Since we use LangGraph, we are already building "provider-agnostic" tools, so what's the point?

Well, the point is that we can get lots of useful tools, prompts, resources, already built from others. Again, no need to reinvent the wheel if we want to dotate our agent with a tool to work on Github.  

Here there is a huge list of MCP servers divided into categories: [MCP Servers](https://github.com/modelcontextprotocol/servers). 

Resources: 
- [LangChain YT: Understanding MCP From Scratch](https://youtu.be/CDjjaTALI68?si=nyFegYEL_cTVUX5W)
- [LangChain YT: Using MCP with LangGraph agents](https://youtu.be/OX89LkTvNKQ?si=rQfTh6A3mklaXzbd)

### Example 1: GitHub MCP server 

Let's see how to use an MCP server to get tools:

In [34]:
from langchain_mcp_adapters.client import MultiServerMCPClient  
import os
from dotenv import load_dotenv
load_dotenv()

# 1. Initialize the MCP Client via http (https://github.com/github/github-mcp-server?tab=readme-ov-file)
client = MultiServerMCPClient({
    "github": {
        "transport": "http",
        "url": "https://api.githubcopilot.com/mcp/x/notifications",  # https://github.com/github/github-mcp-server/blob/main/docs/remote-server.md
        "headers": {
                "Authorization": f"Bearer {os.getenv('GITHUB_TOKEN')}"
            },
        
    }
})

# 2. Get the tools
tools = await client.get_tools()  

In [35]:
for tool in tools:
    print(f"{'='*20} {tool.name} {'='*20}\n\n{tool.description}\n")

==================== dismiss_notification ====================

Dismiss a notification by marking it as read or done

==================== get_notification_details ====================

Get detailed information for a specific GitHub notification, always call this tool when the user asks for details about a specific notification, if you don't know the ID list notifications first.

==================== list_notifications ====================

Lists all GitHub notifications for the authenticated user, including unread notifications, mentions, review requests, assignments, and updates on issues or pull requests. Use this tool whenever the user asks what to work on next, requests a summary of their GitHub activity, wants to see pending reviews, or needs to check for new updates or tasks. This tool is the primary way to discover actionable items, reminders, and outstanding work on GitHub. Always call this tool when asked what to work on next, what is pending, or what needs attention in GitHu

Crazy simple right? Look at all the tools we just got out of the box! We will use these in our GitHub agent. 

### Example 2: LangChain MCP server 

Another example: LangChain mcp server to let our agents search directly in the LangChain documentation:

In [21]:
from langchain_mcp_adapters.client import MultiServerMCPClient  

LG_client = MultiServerMCPClient({
    "github": {
        "transport": "http",
        "url": "https://docs.langchain.com/mcp",  
    }
})

# Get the tools
LG_tools = await client.get_tools()  

In [22]:
for tool in LG_tools:
    print(f"{'='*20} {tool.name} {'='*20}\n\n{tool.description}\n")

==================== SearchDocsByLangChain ====================

Search across the Docs by LangChain knowledge base to find relevant information, code examples, API references, and guides. Use this tool when you need to answer questions about Docs by LangChain, find specific documentation, understand how features work, or locate implementation details. The search returns contextual content with titles and direct links to the documentation pages.



In [29]:
# let's use it

from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
load_dotenv()

prompt="""
You are an AI assistant specialized in LangChain Documentation. 
You possess a tool to search into the documentation. 
You can use this several times to improve your knowledge base before answering the user's question. 
"""

docs_agent = create_agent(
    model=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    tools=LG_tools,
    system_prompt=prompt
)

res = await docs_agent.ainvoke({"messages" : [HumanMessage(content="How should i make my documents (let's say they are pdfs) into a vector store?")]})

for message in res['messages']:
    print(message.content)

How should i make my documents (let's say they are pdfs) into a vector store?

[{'type': 'text', 'text': 'Title: Vector stores\nLink: https://docs.langchain.com/oss/javascript/integrations/vectorstores/index\nContent: Overview\nA vector store stores embedded data and performs similarity search. flowchart LR\n\n    subgraph "📥 Indexing phase (store)"\n        A[📄 Documents] --> B[🔢 Embedding model]\n        B --> C[🔘 Embedding vectors]\n        C --> D[(Vector store)]\n    end\n\n    subgraph "📤 Query phase (retrieval)"\n        E[❓ Query text] --> F[🔢 Embedding model]\n        F --> G[🔘 Query vector]\n        G --> H[🔍 Similarity search]\n        H --> D\n        D --> I[📄 Top-k results]\n    end\n\nAdding documents\nYou can add documents to the vector store by using the addDocuments function. import { Document } from "@langchain/core/documents";\nconst document = new Document({\n  pageContent: "Hello world",\n});\nawait vectorStore.addDocuments([document]);\n\nDeleting documents\nYou 